In [2]:
%pip install numpy 
%pip install gym 
%pip install moviepy
%pip install tensorflow 
%pip install matplotlib
%pip install tdqm

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for tdqm: filename=tdqm-0.0.1-py3-none-any.whl size=1334 sha256=4b942101ec0acd9d3c473e0f8596161dd6d31041d1cf196dd1ee3be42cfa1487
  Stored in directory: c:\users\richa\appdata\local\pip\cache\wheels\af\02\71\aae0f7ee738abf19498353918ddae0f90a0d6ceb337b0bbc91
Successfully built tdqm
Note: you may need to restart the kernel to use updated packages.


In [1]:
import numpy as np
import gym
import setuptools.dist
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Input
from tensorflow.keras.optimizers import Adam
from matplotlib import pyplot as plt

In [2]:
class DDQNAgent:
    def __init__(self, state_size, action_size):
        self.n_actions = action_size
        self.lr = 0.05
        self.gamma = 0.99
        self.epsilon = 1.0
        self.epsilon_decay = 0.005
        self.memory_buffer = list()
        self.memory_buffer_size = 2000
        self.q_model = self.build_model(state_size, action_size)
        self.q_target_model = self.build_model(state_size, action_size)

    def build_model(self, state_size, action_size):
        model = Sequential([
            Input(shape=(state_size,)),
            Dense(units=24, activation='relu'),
            Dense(units=24, activation='relu'),
            Dense(units=action_size, activation='linear')
        ])
        model.compile(loss='mse', optimizer=Adam(learning_rate=self.lr))
        return model
    
    def compute_action(self, current_state):
        if np.random.uniform(0,1) < self.epsilon:
            return np.random.choice(range(self.n_actions))
        else:
            q_values = self.q_model.predict(current_state)[0]
            return np.argmax(q_values)
        
    def store_episode(self, current_state, action, reward, next_state, done):
        self.memory_buffer.append({
            "current_state": current_state,
            "action": action,
            "reward": reward,
            "next_state": next_state,
            "done": done
        })

        if len(self.memory_buffer) >= self.memory_buffer_size:
            self.memory_buffer.pop(0)

    def update_epsilon(self):
        self.epsilon = self.epsilon_decay * self.epsilon

    def train(self, batch_size):
        np.random.shuffle(self.memory_buffer)
        batch_sample = self.memory_buffer[0:batch_size]

        for experience in batch_sample:
            q_current_state = self.q_model.predict(experience["current_state"])[0]

            if experience["done"]:
                q_target = experience["reward"]
            else:
                q_target = experience["reward"] + (self.gamma * np.max(self.q_target_model.predict(experience["next_state"])[0]))

            q_current_state[experience["action"]] = q_target
            self.q_model.fit(experience["current_state"], np.array([q_current_state]), verbose=0)

    def update_q_target_network(self):
        self.q_target_model.set_weights(self.q_model.get_weights())

In [6]:
env = gym.make("CartPole-v1")
state_size = env.observation_space.shape[0]
action_size = env.action_space.n
n_episodes = 100
max_iterations_ep = 400
batch_size = 64
q_target_update_freq = 10

agent = DDQNAgent(state_size, action_size)
total_steps = 0
n_training = 0

In [8]:
for episode in range(n_episodes):
    print(f"Episódio {episode}")
    current_state = env.reset()
    current_state = np.array([current_state[0]])

    rewards = 0

    for step in range(max_iterations_ep):
        total_steps += 1

        action = agent.compute_action(current_state)        
        next_state, reward, done, _, _ = env.step(action)
        next_state = np.array([next_state])

        rewards = rewards + reward
        agent.store_episode(current_state, action, reward, next_state, done)
        
        if done:
            agent.update_epsilon()
            break

        current_state = next_state
    
    print(" rewards: ", rewards)

    if total_steps >= batch_size:
        agent.train(batch_size=batch_size)
        n_training = n_training + 1
        if n_training % q_target_update_freq:
            agent.update_q_target_network()

Episódio 0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
 rewards:  10.0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━

KeyboardInterrupt: 

In [5]:
def make_video():
    import time
    temp_env = gym.make('CartPole-v1', render_mode="rgb_array")
    env = gym.wrappers.RecordVideo(env=temp_env, video_folder='videos', name_prefix="test-video", episode_trigger=lambda x: x % 2 == 0)
    rewards = 0
    steps = 0
    done = False
    state = env.reset()
    state = np.array([state[0]])

    while not done:
        action = agent.compute_action(state)
        state, reward, done, _, _ = env.step(action)
        state = np.array([state])            
        steps += 1
        rewards += reward

    print(rewards)
    env.close()

make_video()

d:\Projetos_Dev\IA\Q-learning\dqn-cart\.venv\Lib\site-packages\gym\wrappers\record_video.py:75: UserWarning: WARN: Overwriting existing videos at d:\Projetos_Dev\IA\Q-learning\dqn-cart\videos folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step


d:\Projetos_Dev\IA\Q-learning\dqn-cart\.venv\Lib\site-packages\gym\utils\passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Moviepy - Building video d:\Projetos_Dev\IA\Q-learning\dqn-cart\videos\test-video-episode-0.mp4.
Moviepy - Writing video d:\Projetos_Dev\IA\Q-learning\dqn-cart\videos\test-video-episode-0.mp4



Moviepy - Done !
Moviepy - video ready d:\Projetos_Dev\IA\Q-learning\dqn-cart\videos\test-video-episode-0.mp4
18.0
